In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [2]:
def starters(row):
    if row.i < 5:
        return True
    return False

In [3]:
def get_box_score(link, table_id, game_id, team):

    response = requests.get(link)

    soup = BeautifulSoup(response.content, 'html.parser')

    tbl = soup.find("table", {"id": table_id})

    df = pd.read_html(str(tbl))[0]
            
    df.columns = pd.MultiIndex.from_tuples([('c', 'd')]*len(df.columns), names=['level_1', 'level_2'])
    df = df.droplevel('level_1', axis=1)
    df.columns = ['Player', 'MP', 'FG', 'FGA', 'FG%', '2P', '2PA', '2P%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']
    df = df.reset_index()
    df.columns = ['i'] + list(df.columns[1:])
    i = df[df.Player=='Reserves'].index
    df = df.drop(i)
    df['Starter'] = df.apply(lambda row: starters(row), axis=1)
    df['game_id'] = game_id
    df['TEAM'] = team
    df = df[df.columns[1:]]
    
    df_ST = df[df.Player=='School Totals']
    
    st = df[df.Player=='School Totals'].index
    df = df.drop(st)
    
    return df, df_ST

In [4]:
df = pd.read_csv('data/bigten_data_w_links.csv', index_col='Unnamed: 0')

In [8]:
df_bs = pd.DataFrame(columns=['Player', 'MP', 'FG', 'FGA', 'FG%', '2P', '2PA', '2P%', '3P', '3PA',
       '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK',
       'TOV', 'PF', 'PTS', 'Starter', 'game_id', 'TEAM'])
df_st = pd.DataFrame(columns=['Player', 'MP', 'FG', 'FGA', 'FG%', '2P', '2PA', '2P%', '3P', '3PA',
       '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK',
       'TOV', 'PF', 'PTS', 'Starter', 'game_id', 'TEAM'])

for index, row in df.iterrows():
    
    if row.link == '-1':
        continue
    
    team = row.TEAM
    if ' ' in team:
        team = team.replace(' ', '-')
        
    team = team.lower()
    
    try:
        df_bs_temp, df_st_temp = get_box_score(row.link, f'box-score-basic-{team}', row.game_id, row.TEAM)
    except Exception as e:
        print(row.TEAM, row.Opponent, row.link, f'box-score-basic-{team}')
        raise e
    
    df_bs = pd.concat([df_bs, df_bs_temp])
    df_st = pd.concat([df_st, df_st_temp])

In [9]:
df_bs

,Player,MP,FG,FGA,FG%,2P,2PA,2P%,3P,3PA,...,TRB,AST,STL,BLK,TOV,PF,PTS,Starter,game_id,TEAM
0,Jeremiah Rivers,26,2,8,.250,2,8,.250,0,0,...,4,4,1,1,4,2,6,True,0,Indiana
1,Maurice Creek,24,7,11,.636,5,7,.714,2,4,...,2,2,1,1,1,2,17,True,0,Indiana
2,Christian Watford,24,4,6,.667,3,5,.600,1,1,...,11,1,1,1,2,3,14,True,0,Indiana
3,Verdell Jones,22,4,12,.333,3,9,.333,1,3,...,1,2,1,1,1,1,9,True,0,Indiana
4,Tom Pritchard,20,5,6,.833,5,6,.833,0,0,...,5,2,0,1,0,2,10,True,0,Indiana
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,Trevor Anderson,5,0,0,NaN,0,0,NaN,0,0,...,0,0,0,0,0,0,0,False,4036,Wisconsin
9,Jordan Davis,0,0,0,NaN,0,0,NaN,0,0,...,0,0,0,0,0,0,0,False,4036,Wisconsin
10,Carter Gilmore,0,0,0,NaN,0,0,NaN,0,0,...,0,0,0,0,0,0,0,False,4036,Wisconsin
11,Justin Taphorn,0,0,0,NaN,0,0,NaN,0,0,...,0,0,0,0,0,0,0,False,4036,Wisconsin


In [10]:
df_st

,Player,MP,FG,FGA,FG%,2P,2PA,2P%,3P,3PA,...,TRB,AST,STL,BLK,TOV,PF,PTS,Starter,game_id,TEAM
17,School Totals,200,34,61,.557,27,47,.574,7,14,...,34,20,7,6,16,22,83,False,0,Indiana
14,School Totals,200,24,54,.444,19,43,.442,5,11,...,38,11,11,1,17,19,69,False,1,Indiana
12,School Totals,200,28,69,.406,25,54,.463,3,15,...,37,13,10,2,20,20,71,False,2,Indiana
11,School Totals,200,24,57,.421,19,44,.432,5,13,...,31,14,8,6,12,27,67,False,3,Indiana
13,School Totals,200,20,57,.351,15,49,.306,5,8,...,40,11,9,1,15,23,66,False,4,Indiana
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9,School Totals,200,28,60,.467,19,36,.528,9,24,...,23,16,7,2,6,21,73,False,3022,Wisconsin
9,School Totals,200,25,54,.463,13,31,.419,12,23,...,23,18,4,4,8,16,75,False,2172,Wisconsin
9,School Totals,200,22,57,.386,15,40,.375,7,17,...,33,11,3,4,12,15,57,False,3023,Wisconsin
16,School Totals,200,31,61,.508,18,34,.529,13,27,...,34,15,5,8,6,11,85,False,4035,Wisconsin


In [11]:
f

In [13]:
df_st.to_csv('data/bigten_school_totals.csv')

In [18]:
set(df.TEAM.values + df.Opponent.values)

{'NebraskaNevada',
 'IowaTexas Christian',
 'Michigan StateBinghamton',
 'Ohio StateLouisiana-Monroe',
 'IndianaRutgers',
 'IllinoisDePaul',
 'Penn StateGeorge Mason',
 'Michigan StateNotre Dame',
 'Michigan StateMiami (FL)',
 'MichiganGeorge Washington',
 'NebraskaHofstra',
 'Michigan StateMassachusetts',
 'IndianaGrand Canyon',
 'PurdueWestern Michigan',
 "MarylandSaint Peter's",
 'RutgersSouthern Methodist',
 'Ohio StateMorgan State',
 'IowaPittsburgh',
 'MinnesotaLehigh',
 'MinnesotaGreen Bay',
 'IllinoisPurdue',
 'NorthwesternSt. Francis (NY)',
 'MichiganHarvard',
 'MarylandWingate',
 'MinnesotaFlorida Atlantic',
 'MarylandEastern Kentucky',
 'PurdueFairfield',
 'RutgersMarist',
 'IowaNorthern Illinois',
 'IndianaPrinceton',
 'IllinoisEvansville',
 'Ohio StateRhode Island',
 'MichiganFlorida State',
 'IllinoisCoppin State',
 'IndianaEastern Michigan',
 'Ohio StateWashington',
 'IowaCoppin State',
 'Ohio StateColgate',
 'PurdueYoungstown State',
 'MichiganBoston College',
 'Nebrask